In [1]:
import cv2
import imutils
import numpy as np
import cupy as cp
import math
import random
from os import listdir
from os.path import isfile, join
import io
from IPython.display import display
from PIL import Image
import time
from skimage.exposure import match_histograms

#random.seed(19)
z_step_counter = 0

%run ./variables.ipynb
%run ./utils.ipynb
%run ../../utils/image_utils.ipynb

ref = cv2.imread("/mnt/nvme-storage/pfauregi/datasets/atlas/ref_img.png", cv2.IMREAD_GRAYSCALE)

In [2]:
def list_images(data_path, diatoms=True, taxon_filter=None):
    # List all images in dataset and putting them in a dict {"key": "imgpath"}
    i = 0
    images_path = []
    for path in data_path:
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.split(".")[1] in ["png", "tif", "tiff", "TIF", "PNG"]:
                    images_path.append((root, file))
                    i+=1
    #for path in data_path: images_path.extend([(path, f) for f in listdir(path) if isfile(join(path, f))])
    #print(images_path)
    images_dict = {}
    selected_taxons = get_selected_taxons()
    for image_path in images_path:
        if len(image_path[1].split('.'))>=2: 
            # defining key
            if diatoms: 
                if taxon_filter is None:
                    key="diatom"
                else:
                    key = image_path[1].split('_')[1]
                    if not key in selected_taxons:
                        key = None
            else: 
                key="mono"
            if not key is None:
                images_dict.setdefault(key,[]).append(image_path)
    print(i,"images found!")
    return images_dict    

def pick_images(images_dict, n_range=[9,12]):
    random.seed()
    # Pick random images and add them to list
    n = random.randint(n_range[0], n_range[1])
    rand_images = []
    for i in range(n):
        # choosing random images
        rand_key = np.random.choice(list(images_dict.keys()))
        rand_indice = np.random.randint(len(images_dict[rand_key]))
        rand_image_path = images_dict[rand_key][rand_indice]
        # loading image
        image_path = join(rand_image_path[0], rand_image_path[1])
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = match_histograms(img, ref, multichannel=False).astype("uint8")
        #img = cv2.normalize(img, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        rand_images.append({"data":img, "taxon":rand_key})
    
    return rand_images

In [3]:
def patchwork(tmp_images, simple_angles = True, size_px = 1000, overlapping=0, starter=None, scale=[5,5]):
    random.seed()
    art_img = (np.ones((size_px, size_px))).astype(np.uint8)
    if starter is None:
        global_patch = np.zeros_like(art_img)
        global_patch_mask_rogn = np.zeros_like(art_img)
    else:
        global_patch = starter[0]
        global_patch_mask_rogn = starter[1]
    annotations = []
    global z_step_counter
    individual_patches = []
    for img_obj in tmp_images:
        w, h = img_obj["data"].shape
        #rand_scale = np.random.randint(10, 20)/100
        rand_scale = (scale[0]+np.random.exponential(scale[1]))/100
        ratio = (rand_scale*size_px)/np.mean([w,h])
        img = resize_img(img_obj["data"], ratio)
        w, h = img.shape
        mask = np.ones_like(img)*255
        mask_rogn = round_rectangle(np.min([w,h])//2, w, h, value=255).astype("uint8")
        # ROTATING
        if simple_angles:
            angle = random.choice((0,90))
        else:
            angle = random.randint(0,360)

        rotated = imutils.rotate_bound(img, angle)
        rotated_mask = imutils.rotate_bound(mask, angle)
        rotated_mask_rogn = imutils.rotate_bound(mask_rogn, angle)
        #PLACING THE IMAGE WITHOUT OVERLAPPING
        overlap_test = float("inf")
        n_stop = 200
        area = w*h
        while overlap_test>area*overlapping and n_stop != 0:
            # TRANSLATING
            px, py = int(rotated.shape[0]/2), int(rotated.shape[1]/2)
            x, y = random.randint(0,size_px-1), random.randint(0,size_px-1) 
            xmin, xmax, ymin, ymax = x-px, x+px, y-py, y+py
            dxmin, dxmax = (0, -xmin)[xmin<0], (0, size_px-1-xmax)[xmax>size_px-1]
            dymin, dymax = (0, -ymin)[ymin<0], (0, size_px-1-ymax)[ymax>size_px-1]
            # PLACING ON TEMPORARY PATCH/MASL
            patch = np.zeros_like(art_img)
            patch_mask = np.zeros_like(art_img)
            patch_mask_rogn = np.zeros_like(art_img)
            patch[xmin+dxmin:xmax+dxmax, ymin+dymin:ymax+dymax] = rotated[dxmin:2*px+dxmax, dymin:2*py+dymax]
            patch_mask[xmin+dxmin:xmax+dxmax, ymin+dymin:ymax+dymax] = rotated_mask[dxmin:2*px+dxmax, dymin:2*py+dymax]
            patch_mask_rogn[xmin+dxmin:xmax+dxmax, ymin+dymin:ymax+dymax] = rotated_mask_rogn[dxmin:2*px+dxmax, dymin:2*py+dymax]
            # Testing if there is overlapping by comparing to global mask
            overlap_test = len(np.nonzero(np.logical_and(patch_mask, global_patch_mask_rogn))[0])
            n_stop -= 1
        if n_stop > 0:
            individual_patches.append({
                "patch": patch.copy(), 
                "patch_mask": patch_mask_rogn.copy(), 
                "center": (int(math.ceil((np.mean([ymin+dymin, ymax+dymax])))), int(math.ceil((np.mean([xmin+dxmin, xmax+dxmax])))))})
            # (erosion to get rid of black edges)
            kernel_size = 3
            kernel = np.ones((kernel_size,kernel_size),np.uint8)
            #patch_mask = cv2.erode(patch_mask,kernel,iterations = 1)
            patch_mask_rogn = cv2.erode(patch_mask_rogn,kernel,iterations = 1)
            # filling global patches
            cv2.copyTo(patch, patch_mask_rogn, global_patch)
            cv2.copyTo(patch_mask_rogn, patch_mask_rogn, global_patch_mask_rogn)
            patch_mask_rogn[patch_mask_rogn>0]=1
            annotations.append({
                "taxon": img_obj["taxon"],
                "xmin": xmin+dxmin,
                "ymin": ymin+dymin,
                "xmax": xmax+dxmax,
                "ymax": ymax+dymax,
                "patch_mask": patch_mask_rogn,
                "z_index": z_step_counter
                
            })
            z_step_counter+=1
    #CREATING FINAL IMAGE
    cv2.copyTo(global_patch, global_patch_mask_rogn, art_img)
    return global_patch, global_patch_mask_rogn, annotations, individual_patches

In [4]:
def fast_img_filling(global_patch, global_patch_mask_rogn, individual_patches, sigma=10e3, verbose=False):
    # IMPROVED img_filling USING CUPY
    final_img = cp.asarray(global_patch)
    acc, accw = cp.zeros_like(final_img).astype(np.float32), cp.zeros_like(final_img).astype(np.float32)
    # Finding contours
    conts, h = cv2.findContours(global_patch_mask_rogn, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # Getting indices
    height=global_patch.shape[0]
    width=global_patch.shape[1]
    if not hasattr(fast_img_filling, 'W'):
        indices = np.indices((height*2,width*2))
        xMap = cp.asarray(indices[0])
        yMap = cp.asarray(indices[1])
        d2 = cp.square(xMap - width) + cp.square(yMap - height)
        fast_img_filling.W = cp.exp(-d2/sigma)
        fast_img_filling.W[fast_img_filling.W<1e-10] = 1e-10
    # Looping
    i = 0
    known = np.concatenate(conts)
    for kp in known:
        # Counter
        if verbose and i%2000==0:
            print(i, "/", len(known))
            i += 1
        # Init
        xkp, ykp = kp[0][1], kp[0][0]
        val = final_img[xkp, ykp]
        # FILLING
        w = fast_img_filling.W[height-ykp:2*height-ykp,width-xkp:2*width-xkp]
        acc += w*val
        accw += w
    acc = cp.divide(acc, accw)
    acc_img = acc.astype(np.uint8)
    
    # sticking the diatoms
    tmp_global_patch_mask_null = cp.asarray(global_patch_mask_rogn)==0
    #final_img[tmp_global_patch_mask_null]=acc_img[tmp_global_patch_mask_null]
    #final_img = cp.asnumpy(acc_img)
    dst = expand(cp.asnumpy(acc_img))
    center = (height//2, width//2)
    for patch in individual_patches:
        ks = np.floor(abs(np.random.normal(0, 3, 1))).astype("uint8")[0]*2+1
        patch_img = cv2.GaussianBlur(patch["patch"],(ks,ks),0)
        src = expand(patch_img)
        mask = patch["patch_mask"]
        center=patch["center"]
        try:
            dst = cv2.seamlessClone(src, dst, mask, center, cv2.NORMAL_CLONE)
        except ValueError:
            print("hey")
    return dst

In [5]:
def main_generator(simple_angles = False, size_px = 1000, verbose=False, overlapping=0, n_diatoms=[9,12], scale_diatoms=[10,10], n_dust=[15,30], scale_dust=[5,10]):
    # Listing images randomly in datasets
    # if images has already been listed, do not list them again to save some performances
    if not hasattr(main_generator, 'images_dict'):
        main_generator.images_dict = list_images(DATASET_PATH, diatoms=True)
        main_generator.images_dict_dust = list_images(DATASET_DUST_PATH, diatoms=False)
    # Picking randomly in images
    rand_images = pick_images(main_generator.images_dict, n_diatoms)
    # Placing them randomly on the artboard
    global z_step_counter
    z_step_counter = 0
    global_patch, global_patch_mask_rogn, annotations, individual_patches = patchwork(rand_images, 
                                                                  simple_angles=simple_angles, 
                                                                  size_px=size_px, 
                                                                  overlapping=overlapping,
                                                                  scale=scale_diatoms) 
    if not n_dust is None: 
        rand_images_dust = pick_images(main_generator.images_dict_dust, n_dust)
        global_patch, global_patch_mask_rogn, annotations_dust, individual_patches_dust = patchwork(rand_images_dust, 
                                                  simple_angles=simple_angles, 
                                                  size_px=size_px, 
                                                  overlapping=1, 
                                                  starter=[global_patch, global_patch_mask_rogn],
                                                  scale=scale_dust) 
        individual_patches.extend(individual_patches_dust)
    
    # Filling the gaps
    final_image = fast_img_filling(global_patch, global_patch_mask_rogn, individual_patches, sigma=10e3, verbose=verbose)
        
    if verbose:
        print("Finished!")
        display(Image.fromarray(np.hstack([global_patch, global_patch_mask_rogn])))
        img_bb = final_image.copy()
        color = (255, 0, 0)
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 1
        for annotation in annotations:
            img_bb = cv2.rectangle(img_bb, (annotation["ymin"], annotation["xmin"]), (annotation["ymax"], annotation["xmax"]), color, 4)
            img_bb = cv2.putText(img_bb,  annotation["taxon"], (annotation["ymin"], annotation["xmin"]), font, fontScale, color, 6, cv2.LINE_AA) 
        display(Image.fromarray(img_bb))
    return final_image, annotations

In [10]:
'''
start=time.time()
random.seed()
final_image, annotations = main_generator(simple_angles = False, 
                                          size_px = 1000,
                                          verbose=False, 
                                          overlapping=0.3, 
                                          n_diatoms=[9,12],
                                          scale_diatoms=[7,5],                                          
                                          n_dust=[25,40],
                                          scale_dust=[3,4])
display(Image.fromarray(final_image))
print(time.time()-start)
'''

'\nstart=time.time()\nrandom.seed()\nfinal_image, annotations = main_generator(simple_angles = False, \n                                          size_px = 1000,\n                                          verbose=False, \n                                          overlapping=0.3, \n                                          n_diatoms=[9,12],\n                                          scale_diatoms=[7,5],                                          \n                                          n_dust=[25,40],\n                                          scale_dust=[3,4])\ndisplay(Image.fromarray(final_image))\nprint(time.time()-start)\n'